In [10]:
import pandas as pd
import numpy as np
dados = pd.read_csv("Manaus.csv", encoding ='ISO-8859-1', sep = ";", low_memory = False )
#dados.columns
# low_memory = False só para tirar o aviso gerado ao ler diferentes dtypes em uma coluna de um arquivo.
# mas talvez seja necessário especificar os tipos de dados de "_srag" e "_evolucao".

In [2]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108351 entries, 0 to 108350
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _idade                108230 non-null  float64
 1   _faixa etária         108351 non-null  object 
 2   _sexo                 107551 non-null  object 
 3   _bairro               106223 non-null  object 
 4   _classificacao        108351 non-null  object 
 5   _comorb_renal         94992 non-null   object 
 6   _comorb_diabetes      95646 non-null   object 
 7   _comorb_imuno         94859 non-null   object 
 8   _comorb_cardio        95698 non-null   object 
 9   _conclusao            64812 non-null   object 
 10  _dt_notificacao       108340 non-null  object 
 11  _taxa                 102551 non-null  object 
 12  _dt_evolucao          39685 non-null   object 
 13  _raca                 83920 non-null   object 
 14  _dt_sintomas          104710 non-null  object 
 15  

In [3]:
len(dados) 

108351

### Considerar apenas os casos confirmados, excluindo todos os demais registros distintos.

In [4]:
dados._classificacao.unique()

array(['Confirmado', 'Descartado', 'Em análise'], dtype=object)

### Selecionando somente casos confirmados

In [5]:
casos_confirmados = dados.query("_classificacao == 'Confirmado'")

### 1. Quantos atributos descrevem cada exemplo? Quais são eles?

In [6]:
casos_confirmados.columns

Index(['_idade', '_faixa etária', '_sexo', '_bairro', '_classificacao',
       '_comorb_renal', '_comorb_diabetes', '_comorb_imuno', '_comorb_cardio',
       '_conclusao', '_dt_notificacao', '_taxa', '_dt_evolucao', '_raca',
       '_dt_sintomas', '_criterio', '_tipo_teste', '_sintoma_garganta',
       '_sintoma_dispneia', '_sintoma_febre', '_sintoma_tosse',
       '_sintoma_outros', '_etnia', '_profiss_saude', '_srag',
       '_se_notificacao', '_distrito', '_bairro_mapa', '_comorb_respiratoria',
       '_comorb_cromossomica', '_comorb_hepatica', '_comorb_neurologica',
       '_comorb_hemato', '_comorb_obessidade', '_origem', '_evolução'],
      dtype='object')

- 36 atributos. 

### 2. Quantos casos confirmados há em Manaus, cumulativamente?

In [7]:
len(casos_confirmados)

36947

### 3. A qual período de tempo a base de dados se refere, isto é, qual o registro mais antigo e qual o mais recente? Leve em conta a data de notificação.

In [8]:
#precisa tratar data
casos_confirmados = pd.DataFrame(casos_confirmados)
casos_confirmados["_dt_notificacao"] = pd.to_datetime(casos_confirmados_data_frame["_dt_notificacao"],)
casos_confirmados = casos_confirmados_data_frame.sort_values(by="_dt_notificacao")
casos_confirmados = casos_confirmados.dropna(subset=["_dt_notificacao"])

primeiro_caso = casos_confirmados["_dt_notificacao"].head(1)
ultimo_caso = casos_confirmados["_dt_notificacao"].tail(1) 

print("Primerio caso", primeiro_caso)
print("Ultimo caso", ultimo_caso)

NameError: name 'casos_confirmados_data_frame' is not defined

Resposta: Desde 1 de abril até 12 de julho de 2020.

### Vamos excluir todos os atributos relativos às comorbidades, sintomas, etnia, profissão, outras datas que não a de notificação, origem e outros que não estiverem envolvidos no contexto do trabalho solicitado. Estes atributos serão considerados irrelevantes para fins de simplificação. Exclua todas as linhas em que houver dados faltantes para os atributos remanescentes.

In [14]:
dados_copia = casos_confirmados.drop(columns=['_comorb_renal', '_comorb_diabetes', '_comorb_imuno',
                                              '_comorb_cardio', '_comorb_cromossomica', '_comorb_hepatica',
                                              '_comorb_neurologica','_comorb_hemato', '_comorb_obessidade',
                                              '_etnia', '_sintoma_garganta', '_sintoma_dispneia', '_sintoma_febre',
                                              '_sintoma_tosse', '_sintoma_outros', '_dt_evolucao', '_dt_sintomas',
                                              '_profiss_saude', '_origem', '_comorb_respiratoria', '_taxa',
                                              '_criterio','_raca', '_srag', '_evolução'])
dados_copia.columns

Index(['_idade', '_faixa etária', '_sexo', '_bairro', '_classificacao',
       '_conclusao', '_dt_notificacao', '_tipo_teste', '_se_notificacao',
       '_distrito', '_bairro_mapa'],
      dtype='object')

In [ ]:
dados_copia.isnull().sum()

In [ ]:
# ver o que é se_notificacao (excluir caso não precise)
#dados_copia = dados_copia.sort_values(by="_se_notificacao")
#dados_copia._se_notificacao

### Alteração dos dados nulos da coluna tipo_teste para desconhecido como forma de evitar perda de dados

In [ ]:
dados_copia.loc[dados_copia['_tipo_teste'].isnull(), ['_tipo_teste']] = 'DESCONHECIDO'

### Removendo linhas e colunas nulas

In [15]:
#excluindo dados nulos
dados_copia = dados_copia.dropna()

In [ ]:
#dados_copia._bairro.unique()
#dados_copia.groupby(['_bairro']).size()
dados_copia.groupby(['_bairro_mapa']).size()
#dados_copia['_bairro_mapa']


### 1. Quantos exemplos e atributos há na base de dados após a limpeza e organização?

In [ ]:
dados_copia.info()

### 2. Qual a porcentagem de indivíduos recuperados em relação ao todo?

In [ ]:
casos_recuperados = dados_copia.query("_conclusao == 'Recuperado'")
total_casos = len(dados_copia)
total_casos_recuperados = len(casos_recuperados)
porcentagem = (total_casos_recuperados / total_casos) * 100

porcentagem

### 3. Os casos acometeram mais indivíduos do sexo masculino ou feminino?

In [ ]:
dados_copia.groupby(['_sexo']).size()

Resposta: Indíviduos do sexo masculino.

### 4. Qual a média e desvio padrão de idade dos indivíduos que contraíram COVID-19? Qual o indivíduo mais jovem e o mais idoso a contraírem tal enfermidade?

In [ ]:
print("media:", casos_confirmados["_idade"].mean())

print("desvio pradao:", casos_confirmados["_idade"].std())

print("maior idade:", max(casos_confirmados["_idade"]))

print("Menor idade:", min(casos_confirmados["_idade"]))


### 5. Qual o bairro com maior incidência de casos?

In [ ]:
dados_copia.groupby(['_bairro']).size().reset_index(name='casos').sort_values(by='casos', ascending=False)

### 6. Quais os três bairros com maior incidência de casos recuperados?


### 7. Quais os tipos de testes efetuados, segundo os dados? Indique os dados de maneira quantitativa  percentual.

In [57]:
test = dados_copia.groupby(['_tipo_teste']).size()

test = pd.DataFrame([test])
test = test.T
test.columns = ['quantitativo']

total_testes = 0
numero_teste = []
resultados = []

for i in test['quantitativo']:    
    total_testes += i
    numero_teste.append(i)

for j in numero_teste:
    resultados.append((j * 100) / total_testes)


test['porcentagem'] =  resultados



test







,quantitativo,porcentagem
_tipo_teste,,
ECLIA IgG,4,0.064371
ELISA IgM,6,0.096556
RT-PCR,1501,24.155134
TESTE RÁPIDO - ANTICORPO,3602,57.965883
TESTE RÁPIDO - ANTÍGENO,1101,17.718056


### 8. Qual taxa de letalidade pode ser calculada a partir do conjunto de dados? Para calcular esta taxa, considere a fração do total de óbitos pelo total de casos;


### 9. Qual o tipo de correlação, mediante coeficiente de correlação de Pearson, entre a idade e o número de casos? Para responder a esta pergunte, agrupe o número de casos por idade e efetue o cálculo de tal coeficiente. Indique, a partir do resultado, a natureza desta correlação, se é positiva ou negativa, e qual sua intensidade.